In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/VQA_Med

In [3]:
!pwd

/home/arunava/VQA_Med


In [4]:
path = '/home/arunava'
#****************************To be Change to reproduce ou result*********************************************

# location of the data and where to store iamge feature image
path_output_chd = path+'/VQA_Med'    

input_vqa_train = 'train_dataset_pickle/mixed_train_dataset_df.pkl'
input_vqa_valid ='valid_dataset_pickle/valid_dataset_df.pkl'

img_feat_train = 'train_dataset_pickle/mixed-train-image-feature.pickle'
img_feat_valid ='valid_dataset_pickle/valid-image-feature.pickle'

input_test = 'test_dataset_pickle/C1_test_dataset_df.pkl'
img_feat_test = 'test_dataset_pickle/test-image-feature.pickle'

In [5]:
# ! pip install transformers

In [6]:
! pip install torchinfo

In [7]:
import easydict
import nltk
import numpy as np
import random
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import BertTokenizer
from transformers import BertModel
import time
import re
from torchinfo import summary
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import bleu_score
import os
import matplotlib.pyplot as plt
import nltk
import copy

/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import json
with open(path+'/VQA_Med/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 


In [9]:
img_feat_vqa = np.load('/home/arunava/VQA_Med/valid_dataset_pickle/valid_dataset_df.pkl',allow_pickle=True)

In [10]:
img_feat_vqa

image_id                                         image_path  \
0     synpic54733  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1     synpic25647  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
2     synpic35681  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
3     synpic39641  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
4     synpic35693  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
...           ...                                                ...   
1995  synpic45828  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1996  synpic60458  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1997  synpic43524  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1998  synpic59568  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   
1999  synpic48459  /home/arunava/VQA_Med_2019_Dataset/Valid/Image...   

                                               Question  \
0     what imaging modality was used to take this im...   
1                            what kind of scan is this?   
2                      was the mri taken with contrast?   
3     is this a t1 weighted, t2 weighted, or flair i...   
4                            is this a noncontrast mri?   
...                                                 ...   
1995                       what is abnormal in the mri?   
1996                     what is abnormal in the x-ray?   
1997              what is most alarming about this mri?   
1998                       what is abnormal in the mri?   
1999              what is most alarming about this mri?   

                                               Answer  labels  
0                                 ct with iv contrast      33  
1                                     xr - plain film       5  
2                                                  no       2  
3                                                  t2      12  
4                                                 yes       3  
...                                               ...     ...  
1995  marked hydrocephalus due to aqueductal stenosis     177  
1996                                 discoid meniscus     177  
1997    choroid plexus neoplasm, papilloma, carcinoma     127  
1998                               multiple sclerosis      62  
1999                           spine,  tethered  cord     177  

[2000 rows x 5 columns]

In [11]:
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1472,
        "INIT_LEARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":10,
    })

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/arunava/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
seed_value = opt.SEED
np.random.seed(seed_value)
random.seed(seed_value)
torch.manual_seed(seed_value) # return <torch._C.Generator object at 0x7f71cdf7a3d0>
torch.cuda.manual_seed(seed_value) 
torch.cuda.manual_seed_all(seed_value)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
 


In [15]:
class BERTokenizer():

    def __init__(self,opt):
        # Load the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.opt = opt
    #pre-process the text data
    def text_preprocessing(self, text):

        # Remove trailing whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        return text


    # Create a function to tokenize a set of texts
    def preprocessing_for_bert(self, data):
        """Perform required preprocessing steps for pretrained BERT.
        @param    data (np.array): Array of texts to be processed.
        @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
        @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                    tokens should be attended to by the model.
        """
        # Create empty lists to store outputs
        input_ids = []
        attention_masks = []
        MAX_LEN = self.opt.MAX_QUESTION_LEN
        # For every sentence...
        for sent in data:
            encoded_sent = self.tokenizer.encode_plus(
                text=self.text_preprocessing(sent),  # Preprocess sentence
                add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
                max_length=MAX_LEN,                  # Max length to truncate/pad
                pad_to_max_length=True,         # Pad sentence to max length
                #return_tensors='pt',           # Return PyTorch tensor
                truncation=True,
                return_attention_mask=True      # Return attention mask
                )
            
            # Add the outputs to the lists
            input_ids.append(encoded_sent.get('input_ids'))
            attention_masks.append(encoded_sent.get('attention_mask'))

        # Convert lists to tensors
        input_ids = torch.tensor(input_ids)
        attention_masks = torch.tensor(attention_masks)

        return input_ids, attention_masks


In [16]:
# Create the question encoder base on  BertClassfier
class BertQstEncoder(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, opt,freeze_bert=True):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        @param    opt: for configuration parameter
        """
        super(BertQstEncoder, self).__init__()
        self.opt = opt
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in= self.opt.BERT_UNIT_NUM

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.bert.eval()
        self.bert_emb = self.bert.embeddings
        
        self.bert_encode_layer1 = (self.bert.encoder.layer)[0]
        self.bert_encode_layer2 = (self.bert.encoder.layer)[1]
        self.bert_encode_layer3 = (self.bert.encoder.layer)[2]
        self.bert_encode_layer4 = (self.bert.encoder.layer)[3]
        self.bert_encode_layer5 = (self.bert.encoder.layer)[4]
        self.bert_encode_layer6 = (self.bert.encoder.layer)[5]
        self.bert_encode_layer7 = (self.bert.encoder.layer)[6]
        self.bert_encode_layer8 = (self.bert.encoder.layer)[7]
        self.bert_encode_layer9 = (self.bert.encoder.layer)[8]
        self.bert_encode_layer10 = (self.bert.encoder.layer)[9]
        self.bert_encode_layer11 = (self.bert.encoder.layer)[10]
        self.bert_encode_layer12 = (self.bert.encoder.layer)[11]

        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """


        # Feed input to BERT

        with torch.no_grad():
            emb_out = self.bert_emb(input_ids, attention_mask)
            # print('emb_out',emb_out.size())
            layer1= self.bert_encode_layer1(emb_out)
            # print('layer1',layer1[0].shape)
            layer2= self.bert_encode_layer2(layer1[0])
            # print('layer2',layer2[0].shape)
            layer3= self.bert_encode_layer3(layer2[0])
            # print('layer3',layer3[0].shape)
            layer4= self.bert_encode_layer4(layer3[0])
            # print('layer4',layer4[0].shape)
            layer5= self.bert_encode_layer5(layer4[0])
            # print('layer5',layer5[0].shape)
            layer6= self.bert_encode_layer6(layer5[0])
            # print('layer6',layer6[0].shape)
            layer7= self.bert_encode_layer7(layer6[0])
            # print('layer7',layer7[0].shape)
            layer8= self.bert_encode_layer8(layer7[0])
            # print('layer8',layer8[0].shape)
            layer9= self.bert_encode_layer9(layer8[0])
            # print('layer9',layer9[0].shape)
            layer10= self.bert_encode_layer10(layer9[0])
            # print('layer10',layer10[0].shape)
            layer11= self.bert_encode_layer11(layer10[0])
            # print('layer11',layer11[0].shape)
            layer12= self.bert_encode_layer12(layer11[0])
            # print('layer12',layer12[0].shape)

        word_question_representation = (layer11[0] +layer12[0])/2
               
        return word_question_representation

In [17]:
class QuestionFeatureExtractionAtt(nn.Module):

    def __init__(self,opt):
        super(QuestionFeatureExtractionAtt,self).__init__()
        self.opt = opt
        self.NUM_QUESTION_GLIMPSE = self.opt.NUM_QUESTION_GLIMPSE #2
                                            #5              #1000
        self.JOINT_EMB_SIZE = self.opt.MFB_FACTOR_NUM * self.opt.MFB_OUT_DIM
        self.Softmax = nn.Softmax(dim=1)
                                                #768                         #2
        self.Linear1_q_proj = nn.Linear(self.opt.BERT_UNIT_NUM * self.opt.NUM_QUESTION_GLIMPSE, self.JOINT_EMB_SIZE)
        
                                                 #0.3
        self.dropout = nn.Dropout(self.opt.BERT_DROPOUT_RATIO)
        self.Conv1_Qatt = nn.Conv2d(self.opt.BERT_UNIT_NUM, self.opt.IMAGE_CHANNEL, 1)
                                            #1472                        #2
        self.Conv2_Qatt = nn.Conv2d(self.opt.IMAGE_CHANNEL,self.opt.NUM_QUESTION_GLIMPSE,1)
    
    def forward(self,qst_encoding):

        self.batch_size = qst_encoding.shape[0]
        qst_encoding = self.dropout(qst_encoding) # torch.Size([64, 768, 17])
        # print('qst_encoding',qst_encoding.shape) 
        qst_encoding_resh = torch.unsqueeze(qst_encoding,3) # torch.Size([64, 768, 17, 1])
        # print('qst_encoding_resh',qst_encoding_resh.shape) 
        qatt_conv1 = self.Conv1_Qatt(qst_encoding_resh) # torch.Size([64, 1472, 17, 1])
        # print('qatt_conv1',qatt_conv1.shape) 
        qatt_relu = F.relu(qatt_conv1) # torch.Size([64, 1472, 17, 1])
        # print('qatt_relu',qatt_relu.shape)
        qatt_conv2 = self.Conv2_Qatt(qatt_relu) # torch.Size([64, 2, 17, 1])
        # print('qatt_conv2',qatt_conv2.shape)
        qatt_conv2 = qatt_conv2.contiguous().view(self.batch_size,2,-1,1)  # torch.Size([64, 2, 17, 1])
        # print('qatt_conv2',qatt_conv2.shape)
        qatt_softmax = self.Softmax(qatt_conv2) # torch.Size([64, 2, 17, 1])
        # print('qatt_softmax',qatt_softmax.shape)
        qatt_softmax = qatt_softmax.view(self.batch_size,2,-1,1) # torch.Size([64, 2, 17, 1])
        # print('qatt_softmax',qatt_softmax.shape)
        qatt_feature_list=[]
        for i in range(self.NUM_QUESTION_GLIMPSE):
            t_qatt_mask = qatt_softmax.narrow(1,i,1)
            # print('t_att_mask',t_qatt_mask.shape) # 
            t_qatt_mask = t_qatt_mask * qst_encoding_resh
            # print('t_att_mask',t_qatt_mask.shape)
            t_qatt_mask = torch.sum(t_qatt_mask,2,keepdim=True)
            # print('t_att_mask',t_qatt_mask.shape)
            qatt_feature_list.append(t_qatt_mask)
            
        # print('qatt_feature_list',len(qatt_feature_list))
        qatt_feature_concat = torch.cat(qatt_feature_list,1)
        # print('qatt_feature_concat',qatt_feature_concat.shape)
        return qatt_feature_concat



In [18]:
class ImageFeatureExtractionAtt(nn.Module):

    def __init__(self,opt):
        super(ImageFeatureExtractionAtt, self).__init__()
        self.opt = opt
        self.MFB_FACTOR_NUM = self.opt.MFB_FACTOR_NUM
        self.MFB_OUT_DIM = self.opt.MFB_OUT_DIM
        self.NUM_IMG_GLIMPSE =self.opt.NUM_IMG_GLIMPSE
        self.IMG_FEAT_SIZE = self.opt.IMG_FEAT_SIZE
        
        self.JOINT_EMB_SIZE = self.opt.MFB_FACTOR_NUM * self.opt.MFB_OUT_DIM
        self.Softmax = nn.Softmax(dim=-1)

        self.Linear1_q_proj = nn.Linear(self.opt.BERT_UNIT_NUM* self.opt.NUM_QUESTION_GLIMPSE, self.JOINT_EMB_SIZE)
        self.Linear_i_proj = nn.Linear(self.opt.IMAGE_CHANNEL*self.opt.NUM_IMG_GLIMPSE, self.JOINT_EMB_SIZE)
        self.Conv_i_proj = nn.Conv2d(self.opt.IMAGE_CHANNEL, self.JOINT_EMB_SIZE, 1)
        

        self.Dropout_M = nn.Dropout(p=self.opt.MFB_DROPOUT_RATIO)

        self.Conv1_Iatt = nn.Conv2d(self.opt.MFB_OUT_DIM, self.opt.IMAGE_CHANNEL, 1) # (1000, 512, 1)
        self.Conv2_Iatt = nn.Conv2d(self.opt.IMAGE_CHANNEL, self.NUM_IMG_GLIMPSE, 1)
    
    def forward(self,img_feature,qstatt_feature):
        '''
        Image Attention with MFB
        '''
        self.batch_size = img_feature.shape[0]
        q_feat_resh = torch.squeeze(qstatt_feature)                              # N x 1536
        i_feat_resh = img_feature.unsqueeze(3)                                   # N x 512 x 196 x 1
        iatt_q_proj = self.Linear1_q_proj(q_feat_resh)                                  # N x 5000
        iatt_q_resh = iatt_q_proj.view(self.batch_size, self.JOINT_EMB_SIZE, 1, 1)      # N x 5000 x 1 x 1
        iatt_i_conv = self.Conv_i_proj(i_feat_resh)                                     # N x 5000 x 196 x 1
        iatt_iq_eltwise = iatt_q_resh * iatt_i_conv
        iatt_iq_droped = self.Dropout_M(iatt_iq_eltwise)                                # N x 5000 x 196 x 1
        iatt_iq_permute1 = iatt_iq_droped.permute(0,2,1,3).contiguous()                 # N x 196 x 5000 x 1
        iatt_iq_resh = iatt_iq_permute1.view(self.batch_size, self.IMG_FEAT_SIZE, self.MFB_OUT_DIM, self.MFB_FACTOR_NUM)
        iatt_iq_sumpool = torch.sum(iatt_iq_resh, 3, keepdim=True)                      # N x 196 x 1000 x 1 
        iatt_iq_permute2 = iatt_iq_sumpool.permute(0,2,1,3)                             # N x 1000 x 196 x 1
        iatt_iq_sqrt = torch.sqrt(F.relu(iatt_iq_permute2)) - torch.sqrt(F.relu(-iatt_iq_permute2))
        iatt_iq_sqrt = iatt_iq_sqrt.reshape(self.batch_size, -1)                           # N x 196000
        iatt_iq_l2 = F.normalize(iatt_iq_sqrt)
        iatt_iq_l2 = iatt_iq_l2.view(self.batch_size, self.MFB_OUT_DIM, self.IMG_FEAT_SIZE, 1)  # N x 1000 x 196 x 1

        iatt_conv1 = self.Conv1_Iatt(iatt_iq_l2)                    # N x 512 x 196 x 1
        iatt_relu = F.relu(iatt_conv1)
        iatt_conv2 = self.Conv2_Iatt(iatt_relu)                     # N x 2 x 196 x 1
        iatt_conv2 = iatt_conv2.view(self.batch_size*self.NUM_IMG_GLIMPSE, -1)
        iatt_softmax = self.Softmax(iatt_conv2)
        iatt_softmax = iatt_softmax.view(self.batch_size, self.NUM_IMG_GLIMPSE, -1, 1)
        iatt_feature_list = []
        for i in range(self.NUM_IMG_GLIMPSE):
            t_iatt_mask = iatt_softmax.narrow(1, i, 1)              # N x 1 x 196 x 1
            t_iatt_mask = t_iatt_mask * i_feat_resh                 # N x 512 x 196 x 1
            t_iatt_mask = torch.sum(t_iatt_mask, 2, keepdim=True)   # N x 512 x 1 x 1
            iatt_feature_list.append(t_iatt_mask)
        iatt_feature_concat = torch.cat(iatt_feature_list, 1)       # N x 1024 x 1 x 1
        iatt_feature_concat = torch.squeeze(iatt_feature_concat)    # N x 1024
        return iatt_feature_concat





In [19]:
class VqaClassifierModel(nn.Module):
    def __init__(self,opt):
        super(VqaClassifierModel,self).__init__()
        self.opt = opt
        self.JOINT_EMB_SIZE = self.opt.MFB_FACTOR_NUM * self.opt.MFB_OUT_DIM
        
        self.MFB_OUT_DIM = self.opt.MFB_OUT_DIM
        self.MFB_FACTOR_NUM = self.opt.MFB_FACTOR_NUM
        NUM_OUTPUT_UNITS = self.opt.NUM_OUTPUT_UNITS


        self.tokenizer = BERTokenizer(self.opt)
        # print('tokenizer',self.tokenizer)
        self.bert_model = BertQstEncoder(self.opt)
        # print('bert_model',self.bert_model)
        
        print(summary(self.bert_model))
        self.qst_feature_att = QuestionFeatureExtractionAtt(self.opt)
        self.img_feature_att = ImageFeatureExtractionAtt(self.opt)
        self.Linear2_q_proj = nn.Linear(self.opt.BERT_UNIT_NUM*self.opt.NUM_QUESTION_GLIMPSE, self.JOINT_EMB_SIZE)
        self.Linear_i_proj = nn.Linear(self.opt.IMAGE_CHANNEL*self.opt.NUM_IMG_GLIMPSE, self.JOINT_EMB_SIZE)

        self.Dropout_M = nn.Dropout(p=self.opt.MFB_DROPOUT_RATIO)

        self.Linear_predict = nn.Linear(self.opt.MFB_OUT_DIM, NUM_OUTPUT_UNITS)

    def forward(self,img,qst):
        self.batch_size = img.shape[0]
        image_feature = img
        input_ids,attention_masks = self.tokenizer.preprocessing_for_bert(qst)
        # print('input_ids',input_ids.shape)
        # print('attention_masks',attention_masks.shape)
        question_feature = self.bert_model(input_ids.to(device),attention_masks.to(device))
        # print('question_feature',question_feature.shape)
        question_feature = question_feature.transpose(1,2)
        # print('question_feature',question_feature.shape)
        q_featatt = self.qst_feature_att(question_feature)
        # print('q_fetatt',len(q_featatt))
        iatt_feature_concat = self.img_feature_att(image_feature,q_featatt)
        # print('iatt_feature_concat',iatt_feature_concat.shape)
        '''
        Fine-grained Image-Question MFB fusion
        '''
        q_feat_resh = torch.squeeze(q_featatt)
        # print('q_feat_resh',q_feat_resh.shape) # torch.Size([64, 1536])
        mfb_q_proj = self.Linear2_q_proj(q_feat_resh)               # N x 5000
        # print('mfb_q_proj',mfb_q_proj.shape) # torch.Size([64, 5000])
        mfb_i_proj = self.Linear_i_proj(iatt_feature_concat)        # N x 5000
        # print('mfb_i_proj',mfb_i_proj.shape) # torch.Size([64, 5000])
        mfb_iq_eltwise = torch.mul(mfb_q_proj, mfb_i_proj)          # N x 5000
        # print('mfb_iq_eltwise',mfb_iq_eltwise.shape) # torch.Size([64, 5000])
        mfb_iq_drop = self.Dropout_M(mfb_iq_eltwise)
        # print('mfb_iq_drop',mfb_iq_drop.shape) # torch.Size([64, 5000])
        mfb_iq_resh = mfb_iq_drop.view(self.batch_size, 1, self.MFB_OUT_DIM, self.MFB_FACTOR_NUM)   # N x 1 x 1000 x 5
        # print('mfb_iq_resh',mfb_iq_resh.shape) # torch.Size([64, 1, 1000, 5])
        mfb_iq_sumpool = torch.sum(mfb_iq_resh, 3, keepdim=True)    # N x 1 x 1000 x 1
        # print('mfb_iq_sumpool',mfb_iq_sumpool.shape) # torch.Size([64, 1, 1000, 1])
        mfb_out = torch.squeeze(mfb_iq_sumpool)                     # N x 1000
        # print('mfb_out',mfb_out.shape) # torch.Size([64, 1000])
        mfb_sign_sqrt = torch.sqrt(F.relu(mfb_out)) - torch.sqrt(F.relu(-mfb_out))
        # print('mfb_sign_sqrt',mfb_sign_sqrt.shape) # torch.Size([64, 1000])
        mfb_l2 = F.normalize(mfb_sign_sqrt)
        # print('mfb_l2',mfb_l2.shape) # torch.Size([64, 1000])
        prediction = self.Linear_predict(mfb_l2)
        # print('prediction',prediction.shape) # torch.Size([64, 178])
        prediction = F.log_softmax(prediction, -1) # N x num_class
        # print('prediction',prediction.shape) # torch.Size([64, 178])
        return prediction
        


In [20]:
class VqaDataset(data.Dataset):
    '''
        Main class use to retrieve our dataset from pickle file.
    '''

    def __init__(self, input_dir, input_vqa, img_feat_vqa, transform=None, phase = 'train'):
        # print('vqa_path',input_dir+'/'+input_vqa)
        # print('feat_path',input_dir+'/'+img_feat_vqa)
        self.input_dir = input_dir
        self.vqa = np.load(input_dir+'/'+input_vqa, allow_pickle=True )
        self.img_feat_vqa = np.load(input_dir+'/'+img_feat_vqa, allow_pickle=True )
        self.vocab_size = None
        self.phase = phase
        # print('vqa',self.vqa)
        # print('img_feat_vqa',type(self.img_feat_vqa))
   

    def __getitem__(self, idx):

        vqa = self.vqa
        img_feat_vqa = self.img_feat_vqa
        image_id = vqa['image_id'].values[idx]
        image_feat = torch.Tensor(img_feat_vqa[image_id])
        sample = { 'image_feature':image_feat ,  } 
        if (self.phase == 'train'):
            ans2idc_a = vqa['labels_a'].values[idx]
            ans2dic_b = vqa['labels_b'].values[idx]
            answer_a = vqa['answer_a'].values[idx]
            answer_b = vqa['answer_b'].values[idx]
            question_a = vqa['question_a'].values[idx]
            question_b = vqa['question_b'].values[idx]
            sample['label_a'] = ans2idc_a
            sample['label_b'] = ans2dic_b
            sample['answer_a'] = answer_a
            sample['answer_b'] = answer_b
            sample['question_a'] = question_a
            sample['question_b'] = question_b
            sample['lamda'] = vqa['lamda'].values[idx]

        elif (self.phase == 'valid'):
            qst2dic = vqa['Question'].values[idx]  
            answer = vqa['Answer'].values[idx]
            label = vqa['labels'].values[idx]
            sample['question'] = qst2dic
            sample['answer'] = answer
            sample['label'] = label
        
        else:
            sample['iamge_id'] = image_id
            
            
        return sample

    def __len__(self):

        return len(self.vqa)

In [21]:
def get_loader(input_dir, input_vqa_train, input_vqa_valid, img_feat_train, img_feat_valid, batch_size, num_workers,size=228):
    '''
        Load our dataset with dataloader for the train and valid data
    '''

    vqa_dataset = {
        'train': VqaDataset(
            input_dir=input_dir,
            input_vqa=input_vqa_train,
            img_feat_vqa=img_feat_train,
            phase = 'train'),
        'valid': VqaDataset(
            input_dir=input_dir,
            input_vqa=input_vqa_valid,
            img_feat_vqa=img_feat_valid,
            phase = 'valid')}
    

    data_loader = {
        phase: torch.utils.data.DataLoader(
            dataset=vqa_dataset[phase],
            batch_size=batch_size,
            # shuffle=True,
            num_workers=num_workers,
            )
        for phase in ['train','valid']}

    return data_loader

In [22]:
class LabelSmoothingCrossEntropy(torch.nn.Module):
    def __init__(self, eps=0.1, reduction='mean', classes=330):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.eps = eps
        self.reduction = reduction

    def forward(self, output, target):
        B, c = output.size()
        print('B',B)
        print('C',c)
        # log_preds = F.log_softmax(output, dim=-1)
        log_preds = output
        if self.reduction == 'sum':
            loss = -log_preds.sum()
        else:
            loss = -log_preds.sum(dim=-1)
            if self.reduction == 'mean':
                loss = loss.mean()
        return loss * self.eps / c + (1 - self.eps) * F.nll_loss(log_preds, target, reduction=self.reduction)
        

In [23]:
since = time.time()

best_acc1 = 0.0
best_acc5 = 0.0

best_epoch = 0
list_train_loss_per_epoch = []
list_valid_loss_per_epoch = []

list_train_acc1_per_epoch = []
list_valid_acc1_per_epoch = []

model = VqaClassifierModel(opt=opt).to(device)
# criterian = nn.CrossEntropyLoss()
criterian = LabelSmoothingCrossEntropy()
optimizer = optim.Adam(model.parameters(),lr=opt.INIT_LEARNING_RATE,weight_decay=opt.LAMNDA)
input_dir = path_output_chd

saved_dir = path_output_chd
num_epochs = opt.NUM_EPOCHS
batch_size = opt.BATCH_SIZE
# print('batch_size',batch_size)
image_size = opt.IMG_INPUT_SIZE
num_workers = 0
batch_size = opt.BATCH_SIZE

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Layer (type:depth-idx)                                  Param #
BertQstEncoder                                          --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─BertEmbeddings: 1-2                                   (recursive)
│    └─Embedding: 2-4                                   (recursive)
│

In [24]:
data_loader = get_loader(
        input_dir = input_dir , 
        input_vqa_train = input_vqa_train, 
        input_vqa_valid = input_vqa_valid,
        img_feat_train = img_feat_train, 
        img_feat_valid = img_feat_valid,
            batch_size = batch_size, 
            num_workers = num_workers,
            size = image_size )

In [25]:

def mixup_criterion_vqa(criterion, pred_a, pred_b, a_a, a_b, lam):
    # print(lam * criterion(pred_a, a_a))
    # print((1 - lam) * criterion(pred_b, a_b))
    # print(lam * criterion(pred_a, a_a) + (1 - lam) * criterion(pred_b, a_b))
    return lam * criterion(pred_a, a_a) + (1 - lam) * criterion(pred_b, a_b)


In [26]:
def accuracy(output, target_a, target_b, lam, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    
    maxk = max(topk)
    batch_size = target_a.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    
    if target_a.dim() == 2: # multians option
        _, target = torch.max(target, 1)
    correct_a = pred.eq(target_a.view(1,-1).expand_as(pred))
    correct_b = pred.eq(target_b.view(1,-1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = lam*correct_a[:k].reshape(-1).float().sum(0) + (1-lam) * correct_b[:k].reshape(-1).float().sum(0)
        res.append((correct_k / batch_size))
    return res
        

In [27]:
def accuracy_valid(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
   
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
   

    
    if target.dim() == 2: # multians option
        _, target = torch.max(target, 1)
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append((correct_k / batch_size))
   
    return res
        
 
def get_bleu_score_valid(predicted, true_ans_text):
    path_output_change = path_output_chd
    with open(path_output_change+'/answer_classes.json', 'r') as j:
        answer_classes_dict = json.load(j)
    score = 0.0
    assert (len(predicted) == len(true_ans_text))
    ans_keys = list(answer_classes_dict.keys())
    ans_values = list(answer_classes_dict.values())
    

    for pred, true_ans in zip(predicted, true_ans_text):
        index_ans = ans_values.index(pred)
        
        score += sentence_bleu([true_ans.split(' ')], ans_keys[index_ans].split(' '), smoothing_function=bleu_score.SmoothingFunction().method2)

    return score/len(true_ans_text)

In [28]:
def get_bleu_score(predicted, true_ans_a,true_ans_b,lam):
    path_output_change = path_output_chd
    with open(path_output_change+'/answer_classes.json', 'r') as j:
        answer_classes_dict = json.load(j)
    score_a = 0.0
    score_b = 0.0
    assert (len(predicted) == len(true_ans_a))
    assert (len(predicted) == len(true_ans_b))
    ans_keys = list(answer_classes_dict.keys())
    ans_values = list(answer_classes_dict.values())
    

    for pred, true_ans in zip(predicted, true_ans_a):
        index_ans = ans_values.index(pred)
        score_a += sentence_bleu([true_ans.split(' ')], ans_keys[index_ans].split(' '), smoothing_function=bleu_score.SmoothingFunction().method2)

    for pred, true_ans in zip(predicted,true_ans_b):
        index_ans = ans_values.index(pred)

        score_b += sentence_bleu([true_ans.split(' ')], ans_keys[index_ans].split(' '), smoothing_function=bleu_score.SmoothingFunction().method2)
    

    score = lam * score_a + (1-lam)* score_b
    
    return score/len(true_ans_a)

In [29]:
def load_checkpoint(model, optimizer, filename=None): 
    # Note: Input model & optimizer should be pre-defined. This routine only updates their states. 
    start_epoch = 0 
    if os.path.isfile(filename): 
        print("=> loading checkpoint '{}'".format(filename)) 
        checkpoint = torch.load(filename) 
        start_epoch = checkpoint['epoch'] 
        model.load_state_dict(checkpoint['state_dict']) 
        optimizer.load_state_dict(checkpoint['optimizer']) 
        print("=> loaded checkpoint '{}' (epoch {})" .format(filename,
                                                            checkpoint['epoch'])) 
    else: print("=> no checkpoint found at '{}'".format(filename)) 
    return model, optimizer, start_epoch
    


In [30]:
def make_plot(history, epoch_max, path_output_chd, type_plot='loss'):
    train = history['train']
    valid = history['valid']
    fig, ax = plt.subplots()
    epochs = range(epoch_max)
    
    
    if type_plot=='loss':
        plt.plot(epochs, train, '-r', lw=2, label='Training loss')
        plt.plot(epochs, valid, '-b',lw=2, label='validation loss')
        plt.legend(borderaxespad=0.)
        plt.title('Training and Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.savefig(path_output_chd+'/imgs/loss.png')
        
    elif type_plot == 'acc1':
        ltrain = []
        lvalid = []
        for i in range(len(train)):
            ltrain.append(train[i].to('cpu').numpy().tolist())
        for i in range(len(valid)):
            lvalid.append(valid[i].to('cpu').numpy().tolist())
    
        plt.plot(epochs, ltrain, '-r', lw = 2, label='Training Top 1 Accuracy')
        plt.plot(epochs, lvalid, '-b', lw = 2, label='validation Top 1 Accuracy')
        plt.legend(borderaxespad=0.)
        plt.title('Training and Validation Top 1 Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Top 1 Accuracy')
        plt.savefig(path_output_chd+'/imgs/acc1.png')

    elif type_plot == 'acc5':
        ltrain = []
        lvalid = []
        for i in range(len(train)):
            ltrain.append(train[i].to('cpu').numpy().tolist())
        for i in range(len(valid)):
            lvalid.append(valid[i].to('cpu').numpy().tolist())
        plt.plot(epochs, ltrain, '-r', lw = 2, label='Training Top 5 Accuracy')
        plt.plot(epochs, lvalid, '-b', lw = 2, label='validation Top 5 Accuracy')
        plt.legend(borderaxespad=0.)
        plt.title('Training and Validation Top 5 Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Top 5 Accuracy')
        plt.savefig(path_output_chd+'/imgs/acc5.png')
    else:
        ltrain = []
        lvalid = []
        for i in range(len(train)):
            ltrain.append(train[i].to('cpu').numpy().tolist())
        for i in range(len(valid)):
            lvalid.append(valid[i].to('cpu').numpy().tolist())
        plt.plot(epochs, ltrain, '-r', lw = 2, label='Training blue')
        plt.plot(epochs, lvalid, '-b', lw = 2, label='validation blue')
        plt.legend(borderaxespad=0.)
        plt.title('Training and Validation blue')
        plt.xlabel('Epochs')
        plt.ylabel('Blue')
        plt.savefig(path_output_chd+'/imgs/blue.png')

    
    
    plt.show()


In [31]:
for epoch in range(opt.NUM_EPOCHS):
    print('Epoch {}/{}'.format(epoch,num_epochs-1))
    print('-'*10)
    for phase in ['train','valid']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        correct = 0.0
        top1_acc=0
        top5_acc=0
        acc_test_f=0
        bleu=0
        batch_step_size = len(data_loader[phase].dataset)/batch_size
        for batch_idx,batch_sample in enumerate(data_loader[phase]):

            # img = batch_sample['image'].to(device)
            image = batch_sample['image_feature'].to(device)

            if phase == 'train':
                question_a = batch_sample['question_a']#.to(device)
                question_b = batch_sample['question_b']
                labels_a = batch_sample['label_a'].to(device)
                labels_b = batch_sample['label_b'].to(device)
                label_answer_a = batch_sample['answer_a']#.to(device)
                label_answer_b = batch_sample['answer_b']
                lam = batch_sample['lamda'][0]

            elif phase == 'valid':
                question = batch_sample['question']
                answer = batch_sample['answer']
                labels = batch_sample['label'].to(device)
                
            optimizer.zero_grad()
            # print('image.shape',type(image))
            # print('questions',len(question_a))
            # print('questions',len(question_b))
            with torch.set_grad_enabled(phase=='train'):

                
                # output_a=model(image,question_a)
                # output_b=model(image,question_b)
                # output = (lam * output_a) + ((1 - lam) * output_b)
                # _,preds = torch.max(output,1)
                # loss = mixup_criterion_vqa(criterian, output_a, output_b, labels_a, labels_b, lam)
                if phase == 'train':
                    output_a=model(image,question_a)
                    output_b=model(image,question_b)
                    output = (lam * output_a) + ((1 - lam) * output_b)
                    _,preds = torch.max(output,1)
                    loss = mixup_criterion_vqa(criterian, output_a, output_b, labels_a, labels_b, lam)
                    loss.backward()
                    optimizer.step()            
                elif phase == 'valid':
                    output = model(image,question)
                    _,preds = torch.max(output,1)
                    loss = criterian(output,labels)


            running_loss += loss.item() 

            if phase == 'train':
                #exact match score
                acc1, acc5 = accuracy(output.data, labels_a.data, labels_b.data, lam, topk=(1, 5))
                top1_acc += acc1
                top5_acc += acc5
                # bleu score
                b = get_bleu_score(preds, label_answer_a,label_answer_b,lam)
                bleu += b

            elif phase == 'valid':
                #exact match score
                acc1, acc5 = accuracy_valid(output.data, labels.data, topk=(1, 5))
                top1_acc += acc1
                top5_acc += acc5
                # bleu score
                b = get_bleu_score_valid(preds, answer)
                bleu += b
            
            if batch_idx % 10 == 0:
                print('| {} SET | Epoch [{:02d}/{:02d}], Step[{:04d}/{:04d}], Loss: {:.4f}, Top 1 Acc: {:.4f}, Top 5 Acc: {:.4f}, Bleu: {:.4f}'.format(phase.upper(), epoch+1, num_epochs, batch_idx, int(batch_step_size), loss.item(), acc1, acc5, b))#Acc: {:.4f},Bleu: {:.4f},acc, b

        epoch_loss = running_loss/batch_step_size
        epoch_acc1 = top1_acc/batch_step_size
        epoch_acc5 = top5_acc/batch_step_size
        
        epoch_blue = bleu/batch_step_size
        # epoch_bleu = 0.0

        #save the loss and accuracy for train and valid
        if phase =='train':
            
            list_train_loss_per_epoch.append(epoch_loss)
            list_train_acc1_per_epoch.append(epoch_acc1)
            
        else:
            
            list_valid_loss_per_epoch.append(epoch_loss)
            list_valid_acc1_per_epoch.append(epoch_acc1)

            history_loss = {'train':list_train_loss_per_epoch, 'valid':list_valid_loss_per_epoch}
            history_acc1 = {'train':list_train_acc1_per_epoch, 'valid':list_valid_acc1_per_epoch}    

        print('{} Loss: {:.4f} Top 1 Acc: {:.4f} Top 5 Acc: {:.4f} Bleu: {:.4f}'.format(
                phase, epoch_loss, epoch_acc1,epoch_acc5, epoch_blue))

        # deep copy the model
        if phase == 'valid' and epoch_acc1 > best_acc1: #or epoch_acc5 > best_acc5 ):
            best_acc1 = epoch_acc1
            best_acc5 = epoch_acc5
            best_epoch = epoch
            best_model_wts = copy.deepcopy(model.state_dict())

        
        




Epoch 0/9
----------


/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


B 64
C 178
B 64
C 178
| TRAIN SET | Epoch [01/10], Step[0000/0199], Loss: 5.1829, Top 1 Acc: 0.0000, Top 5 Acc: 0.0000, Bleu: 0.0000
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
| TRAIN SET | Epoch [01/10], Step[0010/0199], Loss: 5.0180, Top 1 Acc: 0.2344, Top 5 Acc: 0.5781, Bleu: 0.0186
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178
B 64
C 178


KeyboardInterrupt: 

In [ ]:
history_loss = {'train':list_train_loss_per_epoch, 'valid':list_valid_loss_per_epoch}
history_acc1 = {'train':list_train_acc1_per_epoch, 'valid':list_valid_acc1_per_epoch}

time_elapsed = time.time() - since

print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Top 1 Acc: {:4f}, Top 5 Acc: {:4f}'.format(best_acc1,best_acc5))

#plot the loss and accuracy for train and valid
make_plot(history_loss, num_epochs, input_dir, type_plot='loss')
make_plot(history_acc1, num_epochs,input_dir, type_plot='acc1')

# load best model weights
model.load_state_dict(best_model_wts)
model.load_state_dict(best_model_wts)
state = {'epoch': best_epoch, 
        'state_dict': model.state_dict(), 
        'optimizer': optimizer.state_dict(), 
            'loss':epoch_loss,'valid_accuracy': best_acc1}

full_model_path =saved_dir+'/model_state_seed_97.tar'

torch.save(state, full_model_path)